# Chapter 1 — Installation & Configuration

## 1. Quick Intro

This chapter summarizes the **OpenAI Agents SDK Quickstart** and how it fits in the agentic ecosystem.

## What is it?
The **OpenAI Agents SDK** is a lightweight, production-ready toolkit for building agentic apps with a small set of primitives:
**Agents**, **Handoffs**, **Tools**, **Sessions**, **Guardrails**, and **Tracing**.  
The Python Quickstart covers:
- Creating a project & virtual environment  
- Installing `openai-agents`  
- Setting your `OPENAI_API_KEY`  
- Defining your first Agent and adding handoffs  
- Running orchestration via `Runner`  
- Adding a **guardrail**  
- Viewing **execution traces** in the dashboard  

## What does it do?
- **Hello-world to multi-agent**: start simple, then connect agents using **handoffs**  
- **Operational guardrails**: validate inputs/outputs and ensure reliability  
- **Observability**: automatic **tracing** to inspect tool calls and handoffs  
- **Optional flavors**: includes voice/realtime quickstarts for streaming agents  

---

## Where it sits vs. other frameworks

| Framework | Primary focus | Strengths | Typical use cases |
|------------|----------------|------------|------------------|
| **OpenAI Agents SDK (Python)** | Minimal primitives for production agents (agents, tools, handoffs, guardrails, sessions, tracing) | Small API surface, built-in **handoffs**, **guardrails**, and **tracing** | Multi-agent workflows with compliance and observability |
| **LangChain + LangGraph** | Low-level orchestration of long-running, stateful agents | Fine-grained graph/state control, mature ecosystem | Complex, controllable agents at scale |
| **LlamaIndex (Agents)** | Agent patterns integrated with data & indexing stack | Tight RAG/data tooling; easy to connect to vector stores | Knowledge assistants, data-centric agents |
| **CrewAI** | Multi-agent collaboration with roles and memory | Lightweight; “crew” abstraction for teamwork | Task automation, multi-role collaboration |
| **AutoGen (Microsoft)** | Event-driven multi-agent programming | Rich communication patterns (e.g., GroupChat) | Research and conversational multi-agent workflows |



## 2. Getting Started
### 2.1 Install dependencies

**First things first** :

``` python
pip install "openai-agents[litellm]"

```
**What is it?**   
The `litellm` extra installs the **LiteLLM** integration for the OpenAI Agents SDK. With it, your agents can call **non-OpenAI models** (Anthropic, Google Gemini, Mistral, Bedrock, Groq, local/Ollama via LiteLLM, etc.) through a unified interface.

**When To use it?** 
- You want to **mix or migrate** across providers (Anthropic/Gemini/Mistral/Bedrock/etc.) without rewriting agent code. 
- You’re experimenting with **local or alternative backends** (e.g., via LiteLLM/Ollama or Bedrock). 


### 2.2 API Configuration and Model Setup

In [3]:
import os
from dotenv import load_dotenv

# Load env
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  # We use openai's model here
chat_model = "gpt-4.1-nano-2025-04-14"   # We will be using cheaper model as im broke AF
emb_model = "text-embedding-3-small"

### 2.3 Construct Agent 
Before diving deeper, let’s make sure everything is working

In [23]:
from litellm import completion

response = completion(
    model=chat_model, 
    messages=[{ "content": "What's your skills?","role": "user"}], 
    api_base=base_url,
    api_key=api_key
)
print(response.choices[0].message.content)

Hello! I have a wide range of skills, including:

- Answering questions and providing explanations on a variety of topics
- Assisting with writing, editing, and proofreading text
- Generating creative content like stories, poems, and ideas
- Providing summaries and explanations of complex material
- Helping with language learning and translation
- Offering programming help and coding assistance
- Engaging in thoughtful conversations and brainstorming
- Giving recommendations for books, movies, and other resources

Let me know how I can assist you today!


Here's what we got as output : 

"Hello! I have a wide range of skills, including:

- Answering questions and providing explanations on a variety of topics
- Assisting with writing, editing, and proofreading text
- Generating creative content like stories, poems, and ideas
- Providing summaries and explanations of complex material
- Helping with language learning and translation
- Offering programming help and coding assistance
- Engaging in thoughtful conversations and brainstorming
- Giving recommendations for books, movies, and other resources

Let me know how I can assist you today!"

means that our API key and endpoint are set correctly

Now we let **OpenAI Agents SDK** handle the conversation logic.   
We’ll connect the same `LiteLLM` model to an Agent — add some instructions, and let it do the talking!

In [25]:
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel


# 1) (Optional) turn off tracing if you don’t want execution logs collected
set_tracing_disabled(disabled=True)

# 2) Configure a LiteLLM-backed chat model
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# 3) Create an Agent and run it
agent = Agent(name="Assistant", model=llm, instructions="You are a helpful assistant")



# Async (preferred in web/app backends
result = await Runner.run(agent, "Tell me a programmer joke")  # Use as .ipynb file as jupyter notebook does not support asyncio

# Or sync (CLI / quick scripts)
#result = Runner.run_sync(agent, "Tell me a programmer joke")  


print(result.final_output)

Why do programmers prefer dark mode?

Because light attracts bugs!


Here we create an agent that **always replies in English** — even if the user speaks another language.  
Two key ideas:
- **Behavior via `instructions`**: “You only speak English…” gives the agent a strict style rule.
- **Determinism via `ModelSettings`**: `temperature=0.1` makes outputs more **focused and consistent**.

In [26]:
from agents import Agent, ModelSettings


english_agent = Agent(
    name="English agent",
    model=llm, 
    instructions="You only speak English, Even the user is using other laungage",
    model_settings=ModelSettings(temperature=0.1),
)

result = await Runner.run(english_agent, "给我讲个程序员相亲的笑话")
print(result.final_output)

Sure! Here's a programmer-themed matchmaking joke for you:

Why did the programmer go on a blind date?  
Because he heard she had a great "syntax" and wanted to "debug" his love life!


### 2.4  Verbose Logging (stdout) + Logger Levels

sometimes you just want to **see what the SDK is doing**. This snippet enables verbose logs and lets you control how noisy they are.

In [ ]:
from agents import enable_verbose_stdout_logging
import logging

# 1) Print helpful SDK logs to stdout/stderr
enable_verbose_stdout_logging()


# 2) Get the Agents logger (or use "openai.agents.tracing" for tracing-specific logs)
logger = logging.getLogger("openai.agents") # or openai.agents.tracing for the Tracing logger


# 3) Pick ONE level you want (DEBUG > INFO > WARNING)

# To make all logs show up
logger.setLevel(logging.DEBUG)
# To make info and above show up
logger.setLevel(logging.INFO)
# To make warning and above show up
logger.setLevel(logging.WARNING)
# etc

# You can customize this as needed, but this will output to `stderr` by default
logger.addHandler(logging.StreamHandler())

In [29]:
logger = logging.getLogger("openai.agents")
logger.setLevel(logging.DEBUG)

result = await Runner.run(agent, "Say hi and tell me one fun fact about RAG.")
print("FINAL:", result.final_output)


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


...

Resetting current trace


Resetting current trace


Resetting current trace


Resetting current trace


FINAL: Hi! Here's a fun fact about RAG (Retrieval-Augmented Generation): RAG models combine the strengths of both retrieval-based and generative models by retrieving relevant documents from a large corpus to help generate more accurate and contextually aware responses. This approach allows them to produce more informed and factual outputs!


In [30]:
trace_logger = logging.getLogger("openai.agents.tracing")
trace_logger.setLevel(logging.DEBUG)
if not trace_logger.handlers:
    trace_logger.addHandler(logging.StreamHandler())

result = await Runner.run(agent, "Explain what a handoff is in one line.")
print("FINAL:", result.final_output)


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


...

Resetting current trace


Resetting current trace


Resetting current trace


Resetting current trace


FINAL: A handoff is the process of transferring control or responsibility from one person, team, or system to another.
